In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random

import PIL
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import os
import random
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf

from numpy import save
from numpy import load

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow import keras
from tensorflow.keras import Input, Model

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten

In [2]:
X = load('G:/Desktop/intel image classification/keras_train_x_256(3).npy')
y = load('G:/Desktop/intel image classification/keras_train_y_256(3)(all).npy')

In [3]:
x_test = load('G:/Desktop/intel image classification/keras_test_x_256(3).npy')
y_test = load('G:/Desktop/intel image classification/keras_test_y_256(3)(all).npy')

In [4]:
train_dataset = tf.data.Dataset.from_tensor_slices(X)
test_dataset = tf.data.Dataset.from_tensor_slices(x_test)

In [5]:
train_dataset_y = tf.data.Dataset.from_tensor_slices(y)
test_dataset_y = tf.data.Dataset.from_tensor_slices(y_test)

In [6]:
BATCH_SIZE = 2

train_batches = train_dataset.batch(BATCH_SIZE)
test_batches = test_dataset.batch(BATCH_SIZE)

In [7]:
IMG_SHAPE = (256, 256, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

C:\Users\Dindar\anaconda3\lib\site-packages\keras_applications\mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [8]:
len(base_model.layers)

155

In [9]:
base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 257, 257, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 128, 128, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 128, 128, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________

In [10]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier

# block_10_project_BN

In [17]:
%%time
del x
del model
del prediction
x = base_model.get_layer('block_14_project').output
prediction = tf.keras.layers.Flatten()(x)

model = Model(inputs=base_model.input, outputs=prediction)

block_10_project_BN_train = model.predict(train_batches)

block_10_project_BN_test = model.predict(test_batches)

Wall time: 14min 16s
Parser   : 1.31 s


In [18]:
%%time
del sgd
sgd = SGDClassifier(loss='hinge',n_jobs=-1,alpha=0.01)
sgd.fit(block_10_project_BN_train, y)

preds_train = sgd.predict(block_10_project_BN_train)
preds_test = sgd.predict(block_10_project_BN_test)

Wall time: 17.1 s


In [19]:
accuracy_score(y, preds_train), accuracy_score(y_test, preds_test)

(0.9920906370243694, 0.8963333333333333)

In [20]:
%%time
del sgd
sgd = SGDClassifier(loss='hinge',n_jobs=-1)
sgd.fit(block_10_project_BN_train, y)

preds_train = sgd.predict(block_10_project_BN_train)
preds_test = sgd.predict(block_10_project_BN_test)

Wall time: 14.2 s


In [21]:
accuracy_score(y, preds_train), accuracy_score(y_test, preds_test)

(0.9418554938007696, 0.866)

In [16]:
accuracy_score(y, preds_train), accuracy_score(y_test, preds_test)

(0.983326207781103, 0.91)

In [19]:
%%time
del sgd
sgd = SGDClassifier(loss='hinge',n_jobs=-1)
sgd.fit(block_10_project_BN_train, y)

preds_train = sgd.predict(block_10_project_BN_train)
preds_test = sgd.predict(block_10_project_BN_test)

accuracy_score(y, preds_train), accuracy_score(y_test, preds_test)

Wall time: 50.8 s


(0.987031494940858, 0.9163333333333333)

# block_13_project_BN

In [10]:
x = base_model.get_layer('block_13_project_BN').output
prediction = tf.keras.layers.Flatten()(x)

In [11]:
model = Model(inputs=base_model.input, outputs=prediction)

In [12]:
%%time
block_13_project_BN_train = model.predict(train_batches)

In [13]:
%%time
block_13_project_BN_test = model.predict(test_batches)

Wall time: 15.2 s


In [14]:
block_13_project_BN_train.shape

(14034, 10240)

In [17]:
block_13_project_BN_train.dtype

dtype('float32')

In [15]:
block_13_project_BN_test.shape

(3000, 10240)

In [68]:
del sgd

In [69]:
%%time
sgd = SGDClassifier(loss='hinge',n_jobs=-1, alpha=0.01)
sgd.fit(block_13_project_BN_train, y)

Wall time: 9.56 s


SGDClassifier(alpha=0.01, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=-1, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [70]:
preds_train = sgd.predict(block_13_project_BN_train)
preds_test = sgd.predict(block_13_project_BN_test)

In [71]:
accuracy_score(y, preds_train), accuracy_score(y_test, preds_test)

(0.9991449337323642, 0.9003333333333333)

In [72]:
%%time
sgd = SGDClassifier(loss='log',n_jobs=-1, alpha=0.01)
sgd.fit(block_13_project_BN_train, y)

Wall time: 8.85 s


SGDClassifier(alpha=0.01, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,
              n_iter_no_change=5, n_jobs=-1, penalty='l2', power_t=0.5,
              random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [73]:
preds_train = sgd.predict(block_13_project_BN_train)
preds_test = sgd.predict(block_13_project_BN_test)

In [74]:
accuracy_score(y, preds_train), accuracy_score(y_test, preds_test)

(0.9992874447769702, 0.9003333333333333)